# DBSCAN
This notebook is split into three sections with one for every algorithm.
Essentially, the idea is to cluster movies and users so that we can
recommend the entire cluster they belong to, for both users and movies.

Compared to the other algorithms, this means that we will not be able
to give exactly how much they like a given movie, but just give them
the cluster they reside in.

## Initialise PySpark and data

In [ ]:
from random import random
import time

from sklearn.decomposition import PCA
from pyspark.ml.linalg import DenseVector
from pyspark.mllib.random import RandomRDDs
import pyspark.sql
import pyspark
import pandas as pd
from pyspark import SparkContext, SparkConf, SQLContext
from sklearn.cluster import DBSCAN
import os
import numpy as np

from density.slides_dbscan import my_DBSCAN

if os.path.basename(os.getcwd()) == 'density':
    print("Current dir is", os.getcwd())
    print("Changing dir to be in root")
    os.chdir('..')
    print('now in', os.getcwd())

from proposal.useful_tools import pickle_manager

SPARK_CONF = SparkConf()
SPARK_CONF.set("spark.driver.memory", "10g")
SPARK_CONF.set("spark.cores.max", "4")
SPARK_CONF.set("spark.executor.heartbeatInterval", "3600")
SPARK_CONF.setAppName("word2vec")

SPARK_CONTEXT = SparkContext.getOrCreate(SPARK_CONF)
SPARK = SQLContext(SPARK_CONTEXT)

np.random.seed(42)

In [ ]:
# Assume that I can do this
MOVIES: pd.DataFrame = pickle_manager.load_pickle(
    'pickles/sentences.pickle.lz4'
)
print(MOVIES.head())

MOVIES_FEATURES: pd.DataFrame = pickle_manager.load_pickle(
    'pickles/sentence_features.pickle.lz4'
)
print(MOVIES_FEATURES.head())
MOVIES_FEATURES = MOVIES_FEATURES.reset_index().to_numpy(dtype=np.float64)

# Huh apparently we have nan. If we set to 0 then it should be fine
MOVIES_FEATURES[np.isnan(MOVIES_FEATURES)] = 0
print(MOVIES_FEATURES)

pca_maker = PCA(n_components=2)
PCA_DATA = pca_maker.fit_transform(MOVIES_FEATURES)
print(PCA_DATA)

In [ ]:
PCA_DATA = list(zip(*PCA_DATA))
PCA_x, PCA_y = PCA_DATA[0], PCA_DATA[1]
print(PCA_DATA[0])

In [ ]:
import matplotlib.pyplot as plt

plt.scatter(PCA_x, PCA_y, s=0.005)
plt.show()

So we actually have a pretty cool way to represent clusters visually by doing this! Ideally we can break
this into chunks of movies with 100 or so recommendations. In theory, it should not just turn brown, but it
might.

## Finding Optimal Parameters
To be safe with defining epsilon and radius, it was decided that the library
will be used first. This can let us just train for the optimal
epsilon and minimum samples while knowing it works.

If you open a website for movies, you probably want to see at least 100 movies.
Let's assume that's our minimum number of points then.

In [ ]:
# This is probably going to light up as copied from somewhere
MIN_POINTS = 100

from sklearn.neighbors import NearestNeighbors

start = time.perf_counter()
neighbors = NearestNeighbors(n_neighbors=MIN_POINTS, metric='cosine')
neighbors_fit = neighbors.fit(MOVIES_FEATURES)
distances, indices = neighbors_fit.kneighbors(MOVIES_FEATURES)
print("That took", time.perf_counter() - start)

distances = np.sort(distances, axis=0)
distances = distances[:,1]
plt.plot(distances)
plt.title('Number of nearest neighbours with a given distance between points')
plt.xlabel('Number of points')
plt.ylabel('Distance between points')
plt.show()

Our ideal value for epsilon is going to be sitting right at that turn in the centre. Most
people just eye the value, but we can actually just use the second derivative to find this point.

First issue is that this is not a continuous function, so we are going to have to find the
discrete form of how to find the second derivative.

Taking the derivative by first principle:
$$
    \frac{dy}{dx} = \lim_{h \rightarrow 0} \frac{f(x+h) - f(x)}{h}
$$

Now to convert this to discrete values, we know that h does exist and cannot
be infinitely close to 0 anymore. Instead, we are forced to select this distance
by the space between the points, which should be 1 in our case.

For our second derivative, we have to find the derivative of the values around the point
then the derivative of those changes. So we do:

In [ ]:
maximum_curve = 0
maximum_curve_index = 0

# We need to use the point in front and behind so we need
# to start at 1 and end one early
for i in range(500, len(distances) - 1):
    a_to_b = distances[i - 1] - distances[i]
    b_to_c = distances[i + 1] - distances[i]
    second_derivative = b_to_c - a_to_b
    if i == 1:
        maximum_curve = second_derivative
        maximum_curve_index = 1

    if second_derivative > maximum_curve:
        maximum_curve = second_derivative
        maximum_curve_index = i

In [ ]:
print("Maximum curvature occurs at", maximum_curve_index)
epsilon = distances[maximum_curve_index]
print("That distance is", epsilon)

Now that is a lot of possible inflections. Odds are that a lot of them are between 0 and 20,000 and 80,000
for us though.

## Scikit DBSCAN
[Documentation](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.DBSCAN.html#sklearn.cluster.DBSCAN)
### Implementation

In [ ]:
start = time.perf_counter()
scikit_movies_clustering = DBSCAN(
    eps=epsilon, min_samples=MIN_POINTS, n_jobs=-1
).fit(MOVIES_FEATURES)
print("Took", time.perf_counter() - start, "seconds")

In [ ]:
def count_noise(labels):
    count = 0
    for i in labels:
        if i == -1:
            count += 1
    return count


print(scikit_movies_clustering.labels_)
print("Unique labels", set(scikit_movies_clustering.labels_))
print("Number of noise points", count_noise(scikit_movies_clustering.labels_))

print("Frequency table", np.unique(np.unique(scikit_movies_clustering.labels_, return_counts=True)))

    print("==============================")
    print("Doing", i)

    start = time.perf_counter()
    scikit_movies_clustering = DBSCAN(
        eps=i, min_samples=5,  n_jobs=-1, metric='cosine'
    ).fit(MOVIES_FEATURES)
    print("Took", time.perf_counter() - start, "seconds")

    print(scikit_movies_clustering.labels_)
    print("Unique labels", set(scikit_movies_clustering.labels_))
    print("Number of noise points", count_noise(scikit_movies_clustering.labels_))

    print("Frequency table", np.unique(np.unique(scikit_movies_clustering.labels_, return_counts=True)))


## PyClustering DBSCAN
### [Documentation](https://pyclustering.github.io/docs/0.10.1/html/d2/d42/classpyclustering_1_1cluster_1_1dbscan_1_1dbscan.html#details)

In [ ]:
from pyclustering.cluster.dbscan import dbscan

start = time.perf_counter()
pyclustering_movies = dbscan(
    MOVIES_SIMILARITY_MATRIX, MOVIES_RADIUS, MOVIES_MINIMUM_POINTS, True, **{'data_type': 'distance_matrix'}
)
pyclustering_movies.process()
print("Took", time.perf_counter() - start, "seconds")

print("Clusters:", pyclustering_movies.get_clusters())
print("Noise:", pyclustering_movies.get_noise())
print("Number of noise points", len(pyclustering_movies.get_noise()))
print("Frequency table", np.unique(np.unique(pyclustering_movies.get_clusters(), return_counts=True)))

## Slides
### Implementation
Since we made a class that just inherits the scikit dbscan and replaces the fit function, we should
just be able to do the same process here as the scikit section

In [ ]:
start = time.perf_counter()
my_dbscan_movies_clustering = my_DBSCAN(
    eps=MOVIES_RADIUS, min_samples=MOVIES_MINIMUM_POINTS, metric='precomputed', n_jobs=-1
).fit(MOVIES_SIMILARITY_MATRIX.copy())
print("Took", time.perf_counter() - start, "seconds")

In [ ]:
print(set(my_dbscan_movies_clustering.labels_))
print("Number of noise points", count_noise(my_dbscan_movies_clustering.labels_))
print("Frequency table", np.unique(np.unique(my_dbscan_movies_clustering.labels_, return_counts=True)))

## Evaluation

In [ ]:
from sklearn import metrics


def evaluate_clustering_numerically(labels, features):
    """
    All of the prints that are needed to represent how well the data is clustered
    :param labels:
    :return:
    """
    print("Number of clusters", len(set(labels)) - (1 if -1 in labels else 0))
    print("Noise fraction", list(labels).count(-1) / len(labels))
    print("Silhouette Coefficient", metrics.silhouette_score(features, labels))

In [ ]:
evaluate_clustering_numerically(
    scikit_movies_clustering.labels_,
    MOVIES_FEATURES
)

In [ ]:
evaluate_clustering_numerically(
    pyclustering_movies.get_clusters(),
    MOVIES_SIMILARITY_MATRIX
)

In [ ]:
evaluate_clustering_numerically(
    my_dbscan_movies_clustering.labels_,
    MOVIES_SIMILARITY_MATRIX
)